In [1]:
import numpy as np
import pandas as pd

In [34]:
#Load dataset
dfDataChampionship            =  pd.read_csv("campeonato-brasileiro-full.csv", delimiter=";")

#To String
dfDataChampionship            =  dfDataChampionship.applymap(str);  

#Create new column Placar
dfDataChampionship["Placar"]  =  dfDataChampionship["Mandante Placar"].map(str) + "x" + dfDataChampionship["Visitante Placar"]

#Show data
dfDataChampionship.head()

,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Placar
0,1,2000-07-29,16h00,Sábado,Fluminense,Bahia,Fluminense,Maracanã,2,0,RJ,BA,RJ,2x0
1,1,2000-07-29,16h00,Sábado,Vasco,Sport,Sport,São Januário,0,2,RJ,PE,PE,0x2
2,1,2000-07-29,16h00,Sábado,Vitória,Palmeiras,Vitória,Barradão,4,1,ES,SP,ES,4x1
3,1,2000-07-30,17h00,Domingo,Botafogo-RJ,Atlético-MG,-,Caio Martins,0,0,RJ,MG,-,0x0
4,1,2000-07-30,18h30,Domingo,Juventude,Flamengo,-,Alfredo Jaconi,1,1,RS,RJ,-,1x1


In [35]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);

In [36]:
filter1  =  dfDataChampionship['Data']>='2020-01-01'
filter2  =  dfDataChampionship['Data']<='2021-02-26'
brasilianLeague = dfDataChampionship[filter1 & filter2]
#brasilianLeagle2019.to_csv('brasilianLeagle2019.csv', index=None, header=True)
brasilianLeague.head()

,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Placar
7939,1,2020-08-08,19:00,Sábado,Fortaleza,Athlético-PR,Athlético-PR,Castelão,0,2,CE,PR,PR,0x2
7940,1,2020-08-08,19:30,Sábado,Coritiba,Internacional,Internacional,Couto Pereira,0,1,PR,RS,RS,0x1
7941,1,2020-08-08,21:00,Sábado,Sport,Ceará,Sport,Ilha do Retiro,3,2,PE,CE,PE,3x2
7942,1,2020-08-09,16:00,Domingo,Flamengo,Atlético-MG,Atlético-MG,Maracanã,0,1,RJ,MG,MG,0x1
7943,1,2020-08-09,16:00,Domingo,Santos,Bragantino,-,Vila Belmiro,1,1,SP,SP,-,1x1


In [37]:
#Cancatenando as colunas
clubs  =  pd.concat([brasilianLeague['Mandante'].str.lower(), brasilianLeague['Visitante'].str.lower()], axis=1, keys=['Clubes'])


In [38]:
#Obtem todos os clubes 
cb  =  pd.Series(clubs['Clubes'].unique(), name="Clubes")
cb  =  cb.to_frame()
cb

,Clubes
0,fortaleza
1,coritiba
2,sport
3,flamengo
4,santos
5,grêmio
6,botafogo-rj
7,corinthians
8,goiás
9,palmeiras


In [39]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Mandante"].str.lower() == clube
    filter2 = df["Visitante"].str.lower() == clube
    return (data[filter1]['Mandante'].count() + data[filter2]['Visitante'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Mandante"].str.lower() == clube
    filter2 = df["Visitante"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Mandante"].str.lower() == clube) | (df["Visitante"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Mandante"].str.lower() == clube) | (df["Visitante"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Mandante"].str.lower() == clube) | (df["Visitante"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Mandante"].str.lower() == clube
    filter2  =  df["Visitante"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [40]:

dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = 0

def ensureUtf(s, encoding='utf8'):
  if type(s) == bytes:
    return s.decode(encoding, 'ignore')
  else:
    return s

for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    c1  =  ensureUtf( c1 )
    c1  =  c1.strip()
    
    pg       =  getPoints(brasilianLeague, c1 )
    j        =  getNumberDepartures(brasilianLeague, c1 )
    v        =  getNumberWinner(brasilianLeague, c1 )
    e        =  getDraw(brasilianLeague, c1 )
    d        =  getDefeats(brasilianLeague, c1 )
    gp, gc   =  getGP(brasilianLeague, c1 )
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc

dfTable['Clubes'] = dfTable['Clubes'].str.capitalize() 
dfTable  =  dfTable.sort_values(by=['PG'], ascending=False)
dfTable  =  dfTable.reset_index(drop=True)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,Flamengo,71,38,21,8,9,68,48,20
1,Internacional,70,38,20,10,8,61,35,26
2,Atlético-mg,68,38,20,8,10,64,45,19
3,São paulo,66,38,18,12,8,59,41,18
4,Fluminense,64,38,18,10,10,55,42,13
5,Grêmio,59,38,14,17,7,53,40,13
6,Palmeiras,58,38,15,13,10,51,37,14
7,Santos,54,38,14,12,12,52,51,1
8,Bragantino,53,38,13,14,11,50,40,10
9,Athlético-pr,53,38,15,8,15,38,36,2
